In [4]:
import numpy as np

y = np.array(range(20)) # TODO: np.random.choice ??
N =20
K = 3
y = np.random.multinomial(N, [1/K]*K, size=1) # 用来构建C比较合适 C[i]:i类sample的个数
y = np.random.choice(K, N) # range, size/shape, probabilities(optional)
y_pred =np.random.choice(K, N)

print(y)
print(y_pred)

[1 0 0 0 2 1 0 2 2 0 0 1 2 1 2 0 1 2 1 0]
[0 2 0 1 0 0 2 2 1 0 1 2 0 2 1 2 2 2 1 2]


In [14]:
def metrics_compute(Y, Y_pred):
    
    """
    Y : (N, K) 
    Y_pred: (N, K)
    
    """
    #y_pred = np.argmax(Y_pred, axis = 1)
    #y = np.argmax(Y, axis = 1)
    C_size = np.array([sum(y==i) for i in range(np.min(y), np.max(y))])
    
    TP, FP, TN, FN = [], [], [], []
    precision, recall, f1 = [], [], []
    
    # get tp, fp, tn, fn for each class
    for i in range(len(C_size)):
        tp = np.sum(y_pred[y==i]==i)
        fp = np.sum(y_pred[y!=i]==i)
        tn = np.sum(y_pred[y!=i]!=i)
        fn = np.sum(y_pred[y==i]!=i)
        
        TP.append(tp)
        FP.append(fp)
        TN.append(tn)
        FN.append(fn)
    
    TP, FP, TN, FN = np.array(TP), np.array(FP), np.array(TN), np.array(FN)
    Precision = TP/(TP+FP)
    Recall = TP/C_size
    F1 = Precision*Recall/(Precision+Recall)
    print(Precision, Recall, F1)

metrics_compute(y, y_pred)

[0.33333333 0.2       ] [0.25       0.16666667] [0.14285714 0.09090909]


In [12]:
C = [y[y==i] for i in range(y.min(), y.max()+1)]
"""
[array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 1, 1]),
 array([2, 2, 2, 2, 2, 2])]
"""
C_size = [len(y[y==i]) for i in range(y.min(), y.max()+1)]
C_pred_size = [len(y_pred[y_pred==i]) for i in range(y.min(), y.max()+1)]
print(C_size)
print(C_pred_size)
TP, FP = [], []
for i in range(len(C_size)):
    TP.append(np.sum(y_pred[y==i]==i))
    FP.append(np.sum(y_pred[y!=i]==i))

print(TP, FP)

[8, 6, 6]
[6, 5, 9]
[2, 1, 2] [4, 4, 7]


In [ ]:
import numpy as np
# $y = \frac{1}{1 + e^{WX+B}}$

class LogisticRegression:
    
    def __init__(self, X=None, Y=None,penalty = "L2", _intercept=True) -> None:
        self.W = None
        self.intercept = _intercept
        self.is_fit = False
        self.X = X
        self.Y = Y
        self.gamma = 0 # scaler of penalty
        self.penalty = penalty

    @staticmethod
    def sigmoid(x):
        # x: (N,)
        return 1/(1+ np.exp(-x)) # broadcast

    def predict(self, X): # input: X
        """
        X: (N,M)
        """
        # X -> (N, M+1)

        if not self.W:
            if self.intercept:
                X = np.c_[np.ones(X.shape[0]), X] # 在最左端加了一列，全是1， 这样W是(M+1,) 第一列是B
                self.W = np.random.rand(X.shape[1]+1)
            else:
                self.W = np.random.rand(X.shape[1])
        
        if not self.is_fit and (self.X and self.Y):
            self.fit(self.X, self.Y, iter_max=100, epsilon=0.02)
        
        return LogisticRegression.sigmoid(X*self.W) # -> X@ self.W

    def fit(self, X, Y, iter_max, epsilon):
        """
        X: (N,M)
        Y: (N,)
        W: (M,)
        """
        self.is_fit = True
        # get y_predict
        y_pred = self.predict(X)

        # get loss function
        order = 1 if self.penalty=="L1" else 2
        # TODO review penalty
        penalty = 1/2 * np.sqrt(np.linalg.norm(self.W, ord=order))

        # get gradient

        # update W

    def logloss(self, X, Y_pred, Y_ground):
        """
        X: (N,M)
        Y: (N,)
        """

        N = X.shape[0]
        loss = -(np.log(Y_pred[Y_ground==1]).sum(axis=0) + np.log(1-Y_pred[Y_ground==0]).sum(axis=0))
        loss = -(np.sum(np.log(Y_pred[Y_ground==1]),axis=0) + np.sum(np.log(1-Y_pred[Y_ground==0]),axis=0))
        return loss / N
    
    def _penalty_gradient(self, X, Y_pred, Y_ground):
        """
        if penalty == "L1":
            return self.gamma
        if penalty == "L2":
            return np.sum(np.abs(self.W))
        """
        # TODO review : datetime
        d_penalty = self.gamma * self.W if self.penalty == "l2" else self.gamma * np.sign(self.W)
        return d_penalty

    def _gradient_logloss(self, X, Y_pred, Y_ground):
        # gradient of logloss NN->logit->softmax->cross entropy (logloss)
        # (f(x_i) - y_i) \cdot x_i
        # Y_pred, Y_ground: (N, K) 不是 K logistic regression是二分类问题 (N,)
        # X: (N, M+1)
        # Y: (N,)

        loss_gradient = X.T@(Y_pred - Y_ground) # TODO review: datetime
        return loss_gradient
    
    def gradient(self, X, Y, Y_pred, Y_ground):

        N = len(X)
        return 1/N*(self._penalty_gradient(X, Y, Y_pred, Y_ground) + self._gradient_logloss(X, Y, Y_pred, Y_ground))



def gradient_Jacobian(X, f):
    """
    X: (M，) -> (M+1,)
    """
    # step 1 function: aver f_loss(X), current point: W

    # step 2 (f_loss((W + deltaW, X) - f_loss((W + deltaW, X)) / 2*deltaW 方向导数


    for idx, val in enumerate(X):


# binary classification


def fit(self, X, y, lr=0.01, tol=1e-7, max_iter=1e7):

    # preparation(optional)
    # initialize parameters
    ---iteration starts -----
    # iteration condition: iter<max_iter, l_prev - loss < tol, ...
    # step 1 Get y_pred
    # step 2 get objective function
        # step 2.1 get loss -> depends on loss function: log loss, MSE, loss function of VAE, ...
        # stpe 2.2 (optional) get penalty -> depends on penalty: l1, l2, ...
    # step 3 get gradient of objective function
        # step 2.1 get gradient of loss function
        # step 2.2 get gradient of penalty
    # step 4 update parameters or converge to return
        W = W - lr * deltaW
    ----iteration ends ----